# Pre process and Import Libraries

In [4]:
from kdm.models import KDMSequentialJointClassModel
import kdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import numpy as np
from keras import metrics
from keras import losses
from keras import optimizers
from keras.models import Model
from keras.layers import Input, Dense
import keras
from pandas import read_csv, DataFrame
import os
os.environ["KERAS_BACKEND"] = "tensorflow"


In [5]:
def annual_growth(row, years):
    min_year = years["min"]
    max_year = years["max"]
    row["Indicator Name"] = row["Indicator Name"] + " - [annual growth %]"
    for year in range(max_year, min_year, -1):
        if not np.isnan(row[str(year)]) and not np.isnan(row[str(year - 1)]):
            row[str(year)] = 100 * (float(row[str(year)]) -
                                    float(row[str(year - 1)])) / abs(float(row[str(year - 1)]))
        else:
            row[str(year)] = np.nan
    row[str(min_year)] = np.nan
    return row


In [6]:

def boundary_str(start, end, tier):
    return f'{tier}: {start:+0,.2f} to {end:+0,.2f}'


def relabel(v, boundaries):
    if v >= boundaries[0][0] and v <= boundaries[0][1]:
        return boundary_str(boundaries[0][0], boundaries[0][1], tier='A')
    elif v >= boundaries[1][0] and v <= boundaries[1][1]:
        return boundary_str(boundaries[1][0], boundaries[1][1], tier='B')
    elif v >= boundaries[2][0] and v <= boundaries[2][1]:
        return boundary_str(boundaries[2][0], boundaries[2][1], tier='C')
    else:
        return np.nan


def relabel_array(v, boundaries):
    if v >= boundaries[0][0] and v <= boundaries[0][1]:
        return [1, 0, 0]
    elif v >= boundaries[1][0] and v <= boundaries[1][1]:
        return [0, 1, 0]
    elif v >= boundaries[2][0] and v <= boundaries[2][1]:
        return [0, 0, 1]
    else:
        return [np.nan]


def get_boundaries(tiers):
    prev_tier = tiers[0]
    boundaries = [(prev_tier[0], prev_tier[prev_tier.shape[0] - 1])]
    for index, tier in enumerate(tiers):
        if index is not 0:
            boundaries.append(
                (prev_tier[prev_tier.shape[0] - 1], tier[tier.shape[0] - 1]))
            prev_tier = tier
    return boundaries


<>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/2s/yq898nms72b5qxqbwz_hx8j80000gn/T/ipykernel_64528/3838281611.py:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if index is not 0:


In [7]:


years = {"min": 1960, "max": 2019}

df_raw = read_csv("./italy-raw-data.csv")
df_raw_growth = DataFrame(data=[row if "growth" in row["Indicator Name"] else annual_growth(
    row, years) for index, row in df_raw.iterrows()])

nodes = ['Pop', 'Urb', 'GDP', 'EC', 'FFEC', 'REC', 'EI', 'CO2', 'CH4', 'N2O']
df_growth = df_raw_growth.transpose().iloc[4:]
df_growth.columns = nodes
TIERS_NUM = 3

In [8]:

new_columns = {}
boundaries_map = {}
for i, content in enumerate(df_growth.items()):
    (label, series) = content
    values = np.sort(
        np.array([x for x in series.tolist() if not np.isnan(x)], dtype=float))
    if values.shape[0] < TIERS_NUM:
        print(f'Error: there are not enough data for label {label}')
        break
    boundaries = get_boundaries(tiers=np.array_split(values, TIERS_NUM))
    new_columns[label] = [relabel(value, boundaries)
                          for value in series.tolist()]
    boundaries_map[label] = boundaries

df = DataFrame(data=new_columns)
df.columns = nodes
df.index = range(years["min"], years["max"] + 1)

In [9]:

def get_joint_distribution(states):
    """
    Compute the joint distribution from a list of one-hot encoded state arrays.

    :param states: A list of arrays, each array is a one-hot encoding of a variable state.
    :return: A numpy array representing the joint distribution.
    """
    # Calculate the joint distribution by outer product and then flatten the result
    joint_dist = np.array(states[0])
    for state in states[1:]:
        joint_dist = np.outer(joint_dist, np.array(state))
    joint_dist = joint_dist.flatten()

    return joint_dist


def process_per_node_parsed(df, inputs, outputs, parsed=False):
    data_input = []
    data_output = []
    index = 0
    for row in df.iterrows():
        dt = row[1]
        inpt = []
        if parsed:
            parsed_inpts = []
            for i in inputs:
                value = relabel_array(dt[i], boundaries_map[i])
                parsed_inpts.append(value)
            inpt = get_joint_distribution(parsed_inpts)
        else:
            for i in inputs:
                # get from df_growth the row index and column i
                inpt.append(df_growth.loc[str(index + 1960), i])

        outpt = []
        parsed_outpts = []
        for i in outputs:
            value = relabel_array(dt[i], boundaries_map[i])
            parsed_outpts.append(value)
        outpt = get_joint_distribution(parsed_outpts)

        # if full input and full output
        if not any(np.isnan(inpt)) and not any(np.isnan(outpt)):
            data_input.append(inpt)
            data_output.append(outpt)
        index += 1
    return data_input, data_output

# Inference CO2

## Process and Output Layers

In [10]:

encoded_size = 3
dim_y = 3
encoder = keras.layers.Identity()
n_comp = 55
sequences = [
    {
        'type': 'merge'
    },
    [{
        #'kernel': kdm.layers.RBFKernelLayer(sigma=0.1, dim=3, trainable=True),
        'kernel': kdm.layers.CosineKernelLayer(),
        'dim_x': 3,
        'dim_y': 3,
        'n_comp': 25
    }],
    [{
        'kernel': kdm.layers.CosineKernelLayer(),
        'dim_x': 3,
        'dim_y': 3,
        'n_comp': 55
    }],
    {
        'type': 'merge'
    },
    [{
        'kernel': kdm.layers.CosineKernelLayer(),
        'dim_x': 9,
        'dim_y': 3,
        'n_comp': 24
    }],
    {
        'type': 'merge'
    },]

## Model

In [72]:
kdm_model = KDMSequentialJointClassModel(encoded_size=encoded_size,
                                         dim_y=dim_y,
                                         encoder=encoder,
                                         n_comp=n_comp,
                                         sequences=sequences,
                                         sigma=0.5)

In [73]:
kdm_model.compile(optimizer=optimizers.Adam(learning_rate=5e-5),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

## Process Data

In [74]:
inputs = [[['Pop', 'Urb', 'GDP']], [['EC'], ['EC']], [
    ['REC', 'FFEC']]]
outputs = [[['EC']], [['REC'], ['FFEC']], [['CO2']]]

In [75]:

full_data = []
for i in range(len(inputs)):
    arr = []
    for j in range(len(inputs[i])):
        data_input, data_output = process_per_node_parsed(
            df_growth, inputs[i][j], outputs[i][j], parsed=(i != 0))
        arr.append((data_input, data_output))
    full_data.append(arr)

In [76]:

idx = 0
for i in range(len(full_data)):

    for j in range(len(full_data[i])):

        data_input, data_output = full_data[i][j]
        data_input = np.array(data_input)
        data_output = np.array(data_output)
        kdm_model.init_components(data_input, data_output,
                                  init_sigma=idx == 0, sigma_mult=0.5, super_index=idx+j, index=0)
    idx += len(full_data[i]) + 1


ValueError: The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(18, 9), Received: value.shape=(24, 9). Target variable: <KerasVariable shape=(18, 9), dtype=float32, path=kdm_layer_39/c_x>

In [77]:

X = []
y = []
y_int = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['CO2'], boundaries_map['CO2'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        X.append(inpt)
        value = get_joint_distribution([value_1])
        y.append(value)
        y_int.append(list(value).index(1))
    index += 1


In [78]:
check_1 = kdm_model.evaluate(np.array(X), np.array(y_int))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.2477  


In [79]:

print('Full Dataset loss:', check_1[0])
print('Full Dataset accuracy:', check_1[1])

Full Dataset loss: 1.0986123085021973
Full Dataset accuracy: 0.2777777910232544


In [80]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y_int, test_size=0.1)

print(len(X_train))
print(len(X_test))

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

48
6


## Train

In [67]:
kdm_model = KDMSequentialJointClassModel(encoded_size=encoded_size,
                                         dim_y=dim_y,
                                         encoder=encoder,
                                         n_comp=n_comp,
                                         sequences=sequences,
                                         sigma=0.5)

In [68]:
kdm_model.compile(optimizer=optimizers.Adam(learning_rate=5e-5),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [81]:
history = kdm_model.fit(
    np.array(X_train),  # Your training data
    np.array(y_train),  # Your training labels
    batch_size=4,
    epochs=300,  # Start with 100 epochs and use early stopping
    verbose=1,  # Detailed logging
    validation_split=0.1,  # Explicit validation data
    shuffle=True  # Shuffle the data
)

Epoch 1/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.4735 - val_loss: 1.0986 - val_sparse_categorical_accuracy: 0.2000
Epoch 2/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0984 - sparse_categorical_accuracy: 0.4955 - val_loss: 1.0987 - val_sparse_categorical_accuracy: 0.2000
Epoch 3/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0978 - sparse_categorical_accuracy: 0.6589 - val_loss: 1.0988 - val_sparse_categorical_accuracy: 0.2000
Epoch 4/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0979 - sparse_categorical_accuracy: 0.4628 - val_loss: 1.0987 - val_sparse_categorical_accuracy: 0.2000
Epoch 5/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0974 - sparse_categorical_accuracy: 0.5254 - val_loss: 1.0987 - val_sparse_categorical_accuracy: 0.2000
Epoch 6/300
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0967 - sparse_categorical_accuracy: 0.5513 - val_loss: 1.0986 - val_sparse_categorical_accuracy: 0.2000
Epoch 7/30

In [82]:

check_2 = kdm_model.evaluate(np.array(X), np.array(y_int))
check_3 = kdm_model.evaluate(np.array(X_test), np.array(y_test))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8131 - sparse_categorical_accuracy: 0.7666 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.8966 - sparse_categorical_accuracy: 0.5000


## Result

In [83]:

print('Full Dataset loss:', check_2[0])
print('Full Dataset accuracy:', check_2[1])


print('Test loss:', check_3[0])
print('Test accuracy:', check_3[1])


Full Dataset loss: 0.8236305713653564
Full Dataset accuracy: 0.7592592835426331
Test loss: 0.8965608477592468
Test accuracy: 0.5


# Inference CH4

In [41]:

encoded_size = 3
dim_y = 3
encoder = keras.layers.Identity()
n_comp = 55
sequences = [
    {
        'type': 'merge'
    },
    [{
        #'kernel': kdm.layers.RBFKernelLayer(sigma=0.1, dim=3, trainable=True),
        'kernel': kdm.layers.CosineKernelLayer(),
        'dim_x': 3,
        'dim_y': 3,
        'n_comp': 25
    }],
    [{
        'kernel': kdm.layers.CosineKernelLayer(),
        'dim_x': 3,
        'dim_y': 3,
        'n_comp': 55
    }],
    {
        'type': 'merge'
    },
    [{
        'kernel': kdm.layers.CosineKernelLayer(),
        'dim_x': 9,
        'dim_y': 3,
        'n_comp': 18
    }],
    {
        'type': 'merge'
    },]

In [42]:
kdm_model = KDMSequentialJointClassModel(encoded_size=encoded_size,
                                         dim_y=dim_y,
                                         encoder=encoder,
                                         n_comp=n_comp,
                                         sequences=sequences,
                                         sigma=0.5)

kdm_model.compile(optimizer=optimizers.Adam(learning_rate=5e-5),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [43]:
inputs = [[['Pop', 'Urb', 'GDP']], [['EC'], ['EC']], [
    ['REC', 'FFEC']]]
outputs = [[['EC']], [['REC'], ['FFEC']], [['CH4']]]

In [46]:

full_data = []
for i in range(len(inputs)):
    arr = []
    for j in range(len(inputs[i])):
        data_input, data_output = process_per_node_parsed(
            df_growth, inputs[i][j], outputs[i][j], parsed=(i != 0))
        arr.append((data_input, data_output))
    full_data.append(arr)

In [37]:

idx = 0
for i in range(len(full_data)):

    for j in range(len(full_data[i])):

        data_input, data_output = full_data[i][j]
        data_input = np.array(data_input)
        data_output = np.array(data_output)
        kdm_model.init_components(data_input, data_output,
                                  init_sigma=idx == 0, sigma_mult=0.5, super_index=idx+j, index=0)
    idx += len(full_data[i]) + 1


In [49]:

X = []
y = []
y_int = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['CH4'], boundaries_map['CH4'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        X.append(inpt)
        value = get_joint_distribution([value_1])
        y.append(value)
        y_int.append(list(value).index(1))
    index += 1


In [50]:
check_1 = kdm_model.evaluate(np.array(X), np.array(y_int))

print('Full Dataset loss:', check_1[0])
print('Full Dataset accuracy:', check_1[1])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.3322  
Full Dataset loss: 1.0986123085021973
Full Dataset accuracy: 0.34210526943206787


In [51]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y_int, test_size=0.1)

print(len(X_train))
print(len(X_test))

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

history = kdm_model.fit(
    np.array(X_train), 
    np.array(y_train),
    batch_size=4,
    epochs=1000,  
    verbose=1,
    validation_split=0.1, 
    shuffle=True
)

check_2 = kdm_model.evaluate(np.array(X), np.array(y_int))
check_3 = kdm_model.evaluate(np.array(X_test), np.array(y_test))


print('Full Dataset loss:', check_2[0])
print('Full Dataset accuracy:', check_2[1])


print('Test loss:', check_3[0])
print('Test accuracy:', check_3[1])


34
4
Epoch 1/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.3702 - val_loss: 1.0985 - val_sparse_categorical_accuracy: 0.5000
Epoch 2/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.3587 - val_loss: 1.0989 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0985 - sparse_categorical_accuracy: 0.5485 - val_loss: 1.0991 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0985 - sparse_categorical_accuracy: 0.3603 - val_loss: 1.0992 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0982 - sparse_categorical_accuracy: 0.3754 - val_loss: 1.0994 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 6/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0984 - sparse_categorical_accuracy: 0.3923 - val_loss: 1.0995 - val_sparse_categorical_accuracy: 0.

# Inference N20

In [52]:

encoded_size = 3
dim_y = 3
encoder = keras.layers.Identity()
n_comp = 55
sequences = [
    {
        'type': 'merge'
    },
    [{
        #'kernel': kdm.layers.RBFKernelLayer(sigma=0.1, dim=3, trainable=True),
        'kernel': kdm.layers.CosineKernelLayer(),
        'dim_x': 3,
        'dim_y': 3,
        'n_comp': 25
    }],
    [{
        'kernel': kdm.layers.CosineKernelLayer(),
        'dim_x': 3,
        'dim_y': 3,
        'n_comp': 55
    }],
    {
        'type': 'merge'
    },
    [{
        'kernel': kdm.layers.CosineKernelLayer(),
        'dim_x': 9,
        'dim_y': 3,
        'n_comp': 18
    }],
    {
        'type': 'merge'
    },]

In [53]:
kdm_model = KDMSequentialJointClassModel(encoded_size=encoded_size,
                                         dim_y=dim_y,
                                         encoder=encoder,
                                         n_comp=n_comp,
                                         sequences=sequences,
                                         sigma=0.5)

kdm_model.compile(optimizer=optimizers.Adam(learning_rate=5e-5),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [54]:
inputs = [[['Pop', 'Urb', 'GDP']], [['EC'], ['EC']], [
    ['REC', 'FFEC']]]
outputs = [[['EC']], [['REC'], ['FFEC']], [['N2O']]]

In [55]:

full_data = []
for i in range(len(inputs)):
    arr = []
    for j in range(len(inputs[i])):
        data_input, data_output = process_per_node_parsed(
            df_growth, inputs[i][j], outputs[i][j], parsed=(i != 0))
        arr.append((data_input, data_output))
    full_data.append(arr)

In [332]:

idx = 0
for i in range(len(full_data)):

    for j in range(len(full_data[i])):

        data_input, data_output = full_data[i][j]
        data_input = np.array(data_input)
        data_output = np.array(data_output)
        kdm_model.init_components(data_input, data_output,
                                  init_sigma=idx == 0, sigma_mult=0.5, super_index=idx+j, index=0)
    idx += len(full_data[i]) + 1


In [56]:

X = []
y = []
y_int = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['N2O'], boundaries_map['N2O'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        X.append(inpt)
        value = get_joint_distribution([value_1])
        y.append(value)
        y_int.append(list(value).index(1))
    index += 1


In [57]:
check_1 = kdm_model.evaluate(np.array(X), np.array(y_int))

print('Test loss:', check_1[0])
print('Test accuracy:', check_1[1])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.3322  
Test loss: 1.0986123085021973
Test accuracy: 0.34210526943206787


In [58]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y_int, test_size=0.1)

print(len(X_train))
print(len(X_test))

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

history = kdm_model.fit(
    np.array(X_train),  # Your training data
    np.array(y_train),  # Your training labels
    batch_size=4,
    epochs=1000,  # Start with 100 epochs and use early stopping
    verbose=1,  # Detailed logging
    validation_split=0.1,  # Explicit validation data
    shuffle=True  # Shuffle the data
)

check_2 = kdm_model.evaluate(np.array(X), np.array(y_int))
check_3 = kdm_model.evaluate(np.array(X_test), np.array(y_test))


print('Full Dataset loss:', check_2[0])
print('Full Dataset accuracy:', check_2[1])


print('Test loss:', check_3[0])
print('Test accuracy:', check_3[1])


34
4
Epoch 1/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.2468 - val_loss: 1.0989 - val_sparse_categorical_accuracy: 0.2500
Epoch 2/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0983 - sparse_categorical_accuracy: 0.5647 - val_loss: 1.0991 - val_sparse_categorical_accuracy: 0.2500
Epoch 3/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0985 - sparse_categorical_accuracy: 0.4024 - val_loss: 1.0994 - val_sparse_categorical_accuracy: 0.2500
Epoch 4/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0983 - sparse_categorical_accuracy: 0.4643 - val_loss: 1.0997 - val_sparse_categorical_accuracy: 0.2500
Epoch 5/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0980 - sparse_categorical_accuracy: 0.4768 - val_loss: 1.1000 - val_sparse_categorical_accuracy: 0.2500
Epoch 6/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0981 - sparse_categorical_accuracy: 0.4719 - val_loss: 1.1002 - val_sparse_categorical_accuracy: 0.2500
Epoch 7/100